In [19]:
using Pkg
Pkg.activate(@__DIR__)
Pkg.instantiate()
using SatellitePlayground
using Plots
using LinearAlgebra
SP = SatellitePlayground

include("nominal_dynamics.jl")
include("rollout.jl")

  Activating project at `~/Downloads/cmu_classes/16745/final_project`


generate_jacobians (generic function with 1 method)

In [20]:
J = SP.default_parameters.J
x0 = SP.initialize_orbit()

steps = 100
dt = 0.1
(states, times) = rollout(x0, steps, dt)
ref_traj = [
    SP.RBState(
        attitude=x0.attitude,
        angular_velocity=x0.angular_velocity,
        position=state.position,
        velocity=state.velocity
    )
    for state in states
    
]
@time As, Bs = generate_jacobians(ref_traj, times, J)

intialized orbit!
[100/100]: norm(ω)=0.986 r=<-7133744.208 -0.005 74746.390> b=<-20.712 -13.837 -14.444> dt=0.100
Simulation complete!


  7.266933 seconds (30.43 M allocations: 1.470 GiB, 8.74% gc time, 99.64% compilation time)


(Any[[1.0 0.0 … 0.0 0.0; 0.0 1.0 … 0.0 0.0; … ; 0.002063414325287574 0.010178084315632711 … 0.9987848450269741 0.02369201315357069; -0.04637186279467337 -0.015143453336058654 … -0.02369201315357069 0.9987848450269741], [1.0 0.0 … 0.0 0.0; 0.0 1.0 … 0.0 0.0; … ; 0.002063414325287574 0.010178084315632711 … 0.9987848450269741 0.02369201315357069; -0.04637186279467337 -0.015143453336058654 … -0.02369201315357069 0.9987848450269741], [1.0 0.0 … 0.0 0.0; 0.0 1.0 … 0.0 0.0; … ; 0.002063414325287574 0.010178084315632711 … 0.9987848450269741 0.02369201315357069; -0.04637186279467337 -0.015143453336058654 … -0.02369201315357069 0.9987848450269741], [1.0 0.0 … 0.0 0.0; 0.0 1.0 … 0.0 0.0; … ; 0.002063414325287574 0.010178084315632711 … 0.9987848450269741 0.02369201315357069; -0.04637186279467337 -0.015143453336058654 … -0.02369201315357069 0.9987848450269741], [1.0 0.0 … 0.0 0.0; 0.0 1.0 … 0.0 0.0; … ; 0.002063414325287574 0.010178084315632711 … 0.9987848450269741 0.02369201315357069; -0.046371862

In [35]:
reduce(min,[rank(A) for A in As])

7

In [36]:
reduce(min,[rank(B) for B in Bs])

3

In [23]:
function hat(ω)
    # TODO: implement the hat function 
    ω̂ = [0 -ω[3] ω[2]
        ω[3] 0 -ω[1]
        -ω[2] ω[1] 0]
    
    return ω̂
end

function L(q)
    # TODO: implement L 
    #_L = zeros(4,4)
    _L = [q[1] -q[2:end]'
        q[2:end] q[1]*I + hat(q[2:end])]
    return _L
end

function R(q)
    # TODO: implement R 
    #_R = zeros(4,4)
    _R = [q[1] -q[2:end]'
        q[2:end] q[1]*I - hat(q[2:end])]
    return _R
end

# TODO: implement H function 
H = zeros(4,3)
H = [0 0 0
    1 0 0
    0 1 0
    0 0 1]

function G(q)
    q_mat = [-q[2] q[1] q[4] -q[3];
             -q[3] -q[4] q[1] q[2];
            -q[4] q[3] -q[2] q[1]]
    _G = [I zeros(3,4);
            zeros(3,3) q_mat]
    return _G
end

G (generic function with 1 method)

In [37]:
N = 5
x0 = ref_traj[1]

# TODO: use TVLQR to generate K's 

# use this for TVLQR tracking cost 
Q_lqr = diagm(ones(6))
Qf_lqr = 10*Q_lqr
R_lqr = 0.01*diagm(ones(3))
nu = 3
nx = 13

K = [zeros(nu,nx) for i=1:N-1]
S = [zeros(nx,nx) for i = 1:N]
S[N]=deepcopy(Qf_lqr)
for j = N-1:-1:1
    xk = ref_traj[j]
    qk = xk.attitude
    Gk = G(qk)
    A = Gk*As[j]*Gk'
    B = Gk*Bs[j]
    K[j] = (R_lqr + B'*S[j+1]*B)\(B'*S[j+1]*A)
    S[j] = Q_lqr + K[j]'*R_lqr*K[j] + (A-B*K[j])\(S[j+1]*(A-B*K[j]))
end

In [38]:
# simulation
Xsim = [zeros(4) for i = 1:N]
Usim = [zeros(1) for i = 1:(N-1)]
Xsim[1] = 1*x0 

# here are the real parameters (different than the one we used for DIRCOL)
# this model mismatch is what's going to require the TVLQR controller to track
# the trajectory successfully. 

# TODO: simulate closed loop system 
for i = 1:(N-1)
    # TODO: add feeback control (right now it's just feedforward)
    
    Usim[i] = clamp.(-K[i]*(Xsim[i]-X_dircol[i]), -10, 10) # update this
    Xsim[i+1] = rk4(params_real, Xsim[i], Usim[i], dt)  
end

LoadError: UndefVarError: X_dircol not defined